In [ ]:
import sagemaker
import boto3 
from sagemaker.image_uris import retrieve
from sagemaker.pytorch import PyTorch, PyTorchModel
import os 
from PIL import Image
from torchvision import transforms
import json
from sagemaker.amazon.amazon_estimator import get_image_uri


In [ ]:
sm_boto3 = boto3.client("sagemaker")

sess = sagemaker.Session()

region = sess.boto_session.region_name



In [ ]:
bucket = os.getenv("BUCKET")
role_arn = os.getenv("ROLE_ARN")

In [ ]:

if not bucket  and role_arn:
    
    raise ValueError("Las variables de entorno de SAGEMAKER no estan configuradas.")

else: print("Variables cargados exitosamente")

In [ ]:
training_data = bucket + "Augmented/"

# Train

In [50]:
hyperparameters={
    'epochs': 10,                       # Aumentado para mejor convergencia
    'batch_size': 48,                   # Balance entre velocidad y estabilidad
    'learning_rate': 0.0003,            # Más bajo para mayor precisión
    'weight_decay': 0.00005,            # Regularización ligera
    'early_stopping': True,
    'class_weights': '"[0.15, 0.85]"',      # Ajuste fino para 300 defectuosas vs 2000 frescas
    'use_sampler': True,                # Crítico para detectar defectos
    'augmentation_intensity': 0.8,      # Máximo aumento para casos raros
    'dropout_rate': 0.4,                # Regularización moderada
    'patience': 10,                     # Más tolerancia para aprendizaje fino
    # 'optimizer': 'AdamW',               # Optimizador recomendado
    # 'scheduler': 'ReduceLROnPlateau',   # Ajuste automático de LR
    'val_split': 0.2,                   # Más datos para validación
    'threshold': 0.85                   # Umbral de confianza para producción
    }

In [51]:
estimator = PyTorch(
    entry_point='train.py',  
    source_dir='src',
    role=role_arn,
    instance_count=1,  
    # instance_type='ml.g4dn.xlarge', 
    instance_type ='ml.m5.large',
    framework_version='1.12.0', 
    py_version='py38',  
    hyperparameters=hyperparameters,
    output_path= bucket + "output",  
    sagemaker_session=sess,
    use_spot_intances=True,
    # max_wait=7200,
    max_run=3600,
)

In [52]:

estimator.fit(training_data, input_mode='FastFile', content_type='application/x-image')


[04/13/25 14:43:02] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=510352;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=326144;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=312142;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=393793;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

[04/13/25 14:43:03] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=619560;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=652685;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating training-job with name:                                       ]8;id=123009;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=242885;file:///home/dev/Documents/Github/pitahaya/.venv/lib/python3.13/site-packages/sagemaker/session.py#1042\1042]8;;\
                             pytorch-training-2025-04-13-19-43-02-545                                              

2025-04-13 19:43:04 Starting - Starting the training job...
2025-04-13 19:43:19 Starting - Preparing the instances for training...
2025-04-13 19:43:41 Downloading - Downloading input data......
2025-04-13 19:45:01 Downloading - Downloading the training image...
2025-04-13 19:45:37 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2025-04-13 19:45:39,863 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-04-13 19:45:39,865 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-04-13 19:45:39,873 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2025-04-13 19:45:39,877 sagemaker_pytorch_container.training INFO     Invoking user training script.
2025-04-13 19:45:40,054 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus instal

In [ ]:
#    hyperparameters = {
#         "num_layers": 18,
#         "image_shape": "3,224,224",
#         "num_classes": 2,
#         "num_training_samples": 2300,
#         "mini_batch_size": 48,
#         "epochs": 40,
#         "learning_rate": 0.0003,
#         "use_pretrained_model": 1
#     }
    
    # Obtener imagen del algoritmo
training_image = get_image_uri(region, 'image-classification')
    
    # Crear estimador
estimator = sagemaker.estimator.Estimator(
    training_image,
    role_arn,
    instance_count=1,
    instance_type='ml.p2.xlarge',
    hyperparameters=hyperparameters
)
    

# Deploy


In [ ]:
model = PyTorchModel(
    model_data=estimator.model_data,
    role=role_arn,
    entry_point="inference.py",
    framework_version='1.13',
    py_version='py38',
    source_dir='code',
)

In [ ]:
# Desplegar el modelo
endpoint_name = 'dragon-fruit-endpoint'

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.xlarge', 
    endpoint_name=endpoint_name
)


# Inference

In [ ]:

sagemaker_runtime = boto3.client('sagemaker-runtime')

In [ ]:
image_path = 'Augmented_Dataset/normal/Fresh_Dragon_Augmented_Data0003.jpg'
image = Image.open(image_path)
image = image.resize((224, 224))  


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización estándar
])


In [ ]:
input_tensor = transform(image).unsqueeze(0) 
payload = json.dumps(input_tensor.numpy().tolist())

In [ ]:
# Hacer la predicción llamando al endpoint
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",  
)

In [ ]:
result = json.loads(response['Body'].read().decode())
print(f"Resultado de la predicción: {result}")

In [ ]:
# Función para visualizar imágenes con predicciones
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)


In [ ]:
# Visualización de algunas imágenes con predicciones
model.eval()
inputs, classes = next(iter(dataloaders['val']))
inputs = inputs.to(device)
classes = classes.to(device)

outputs = model(inputs)
_, preds = torch.max(outputs, 1)

plt.figure(figsize=(15, 15))
for i in range(min(batch_size, len(inputs))):
    ax = plt.subplot(min(batch_size, len(inputs)) // 8, 8, i + 1)
    ax.axis('off')
    ax.set_title(f'predicted: {class_names[preds[i]]}\ntrue: {class_names[classes[i]]}')
    imshow(inputs.cpu().data[i])

plt.show()


In [ ]:
predictor.delete_endpoint()